<a href="https://colab.research.google.com/github/emijon156/dungy/blob/main/dark_dungy_matter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
When collaborating on a Google Colab notebook, the best way to share data is by properly sharing the file on Google Drive itself. Mounting your drive only gives you access to your files, not your collaborators. They must also have permission to access the data file.

Here's how to ensure your friend can view the dataset:

1. Share the Data File on Google Drive
Before your friend can access the file from the mounted drive in Colab, they need to have permission.

Go to the Google Drive folder where you uploaded the data file (.csv, .xlsx, etc.).

Right-click on the data file.

Select Share ➡️ Share.

Enter your friend's email address and make sure to give them "Viewer" access or higher. You can also generate a sharable link and set the permissions to "Anyone with the link can view."

Send the sharing notification to your friend.

2. Friend Must Add a Shortcut to Their Drive
Even after sharing, the file won't automatically appear in your friend's mounted Colab directory. They need to add a shortcut to their own "My Drive."

Tell your friend to go to the "Shared with me" section of their Google Drive.

They should find the data file or the folder you shared.

Right-click on the file and select "Add shortcut to Drive."

They should choose a location in their own "My Drive" (e.g., the Colab Notebooks folder).

3. Your Friend Can Now Mount the Drive in Colab
After creating the shortcut, the data file will be accessible through your friend's mounted Google Drive. They can then run the same code you do to access the file.

In their Colab notebook, they will run the code to mount their drive:

Python

from google.colab import drive
drive.mount('/content/drive')
The data file will now be available at the path of the shortcut they created. They can navigate to it and load it into their notebook for the project.

By following these steps, you can ensure that your friend has the necessary permissions to access the data file from their own Google Drive account, making your collaborative project run smoothly.

You can watch a tutorial on how to share files in Google Colab notebooks.
'''

'\nWhen collaborating on a Google Colab notebook, the best way to share data is by properly sharing the file on Google Drive itself. Mounting your drive only gives you access to your files, not your collaborators. They must also have permission to access the data file.\n\nHere\'s how to ensure your friend can view the dataset:\n\n1. Share the Data File on Google Drive\nBefore your friend can access the file from the mounted drive in Colab, they need to have permission.\n\nGo to the Google Drive folder where you uploaded the data file (.csv, .xlsx, etc.).\n\nRight-click on the data file.\n\nSelect Share ➡️ Share.\n\nEnter your friend\'s email address and make sure to give them "Viewer" access or higher. You can also generate a sharable link and set the permissions to "Anyone with the link can view."\n\nSend the sharing notification to your friend.\n\n2. Friend Must Add a Shortcut to Their Drive\nEven after sharing, the file won\'t automatically appear in your friend\'s mounted Colab dir

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numpy
!pip install astropy
!pip install datascience
!pip install matlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for matlab: filename=matlab-0.1-py3-none-any.whl size=1156 sha256=877e0e6789947d92a3a7df4e6ab18e0f25f7deaa32e047ea84f64d97e7e130eb
  Stored in directory: /root/.cache/pip/wheels/ae/1c/e6/41e5aa7c90f5cfafe0b31e7f52035cf23834d24c5fdb0c9dbd
Successfully built matlab


In [ ]:
from datascience import *

expl_raw = Table.read_table('expl.csv').where('default_flag', 1)
expl_tbl = expl_raw.select('pl_name', 'hostname', 'pl_bmasse', 'ra', 'dec', 'sy_dist')
expl_tbl.show(5)

pl_name,hostname,pl_bmasse,ra,dec,sy_dist
11 Com b,11 Com,4914.9,185.179,17.7933,93.1846
11 UMi b,11 UMi,4684.81,229.275,71.8239,125.321
14 And b,14 And,1131.15,352.824,39.2358,75.4392
14 Her b,14 Her,2559.47,242.602,43.8164,17.9323
16 Cyg B b,16 Cyg B,565.737,295.466,50.5168,21.1397


In [ ]:
ra = expl_tbl.column('ra')
dec = expl_tbl.column('dec')
dist = expl_tbl.column('sy_dist')

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np

coords = SkyCoord(
    ra=np.array(ra) * u.deg,
    dec=np.array(dec) * u.deg,
    distance=np.array(dist) * u.parsec,
    frame='icrs'
)

galactocentric_coords = coords.transform_to('galactocentric')

expl_coord = expl_tbl.with_columns([
    'galactocentric_x', galactocentric_coords.x.value,
    'galactocentric_y', galactocentric_coords.y.value,
    'galactocentric_z', galactocentric_coords.z.value
])

In [ ]:
x_coord = np.round(galactocentric_coords.x.value, 2)
y_coord = np.round(galactocentric_coords.y.value, 2)
z_coord = np.round(galactocentric_coords.z.value, 2)

i = 0
coordinate = []
while i <= expl_tbl.num_rows - 1:
  single_coordinate = f'({x_coord[i]}, {y_coord[i]}, {z_coord[i]})'
  coordinate.append(single_coordinate)
  i += 1

expl_coord = expl_coord.with_column('coordinate', coordinate)
expl_coord.show(5)

pl_name,hostname,pl_bmasse,ra,dec,sy_dist,galactocentric_x,galactocentric_y,galactocentric_z,coordinate
11 Com b,11 Com,4914.9,185.179,17.7933,93.1846,-8123.67,-18.8287,112.047,"(-8123.67, -18.83, 112.05)"
11 UMi b,11 UMi,4684.81,229.275,71.8239,125.321,-8152.1,89.5176,103.169,"(-8152.1, 89.52, 103.17)"
14 And b,14 And,1131.15,352.824,39.2358,75.4392,-8141.94,67.5355,-6.24708,"(-8141.94, 67.54, -6.25)"
14 Her b,14 Her,2559.47,242.602,43.8164,17.9323,-8117.59,11.4422,33.8918,"(-8117.59, 11.44, 33.89)"
16 Cyg B b,16 Cyg B,565.737,295.466,50.5168,21.1397,-8119.57,20.4417,25.6228,"(-8119.57, 20.44, 25.62)"


In [ ]:
r_galactic = np.sqrt(
    np.array(expl_coord.column('galactocentric_x'))**2 +
    np.array(expl_coord.column('galactocentric_y'))**2 +
    np.array(expl_coord.column('galactocentric_z'))**2
)

expl_coord = expl_coord.with_column('r_galactic', r_galactic)
expl_coord.show(5)

pl_name,hostname,pl_bmasse,ra,dec,sy_dist,galactocentric_x,galactocentric_y,galactocentric_z,coordinate,r_galactic
11 Com b,11 Com,4914.9,185.179,17.7933,93.1846,-8123.67,-18.8287,112.047,"(-8123.67, -18.83, 112.05)",8124.47
11 UMi b,11 UMi,4684.81,229.275,71.8239,125.321,-8152.1,89.5176,103.169,"(-8152.1, 89.52, 103.17)",8153.24
14 And b,14 And,1131.15,352.824,39.2358,75.4392,-8141.94,67.5355,-6.24708,"(-8141.94, 67.54, -6.25)",8142.22
14 Her b,14 Her,2559.47,242.602,43.8164,17.9323,-8117.59,11.4422,33.8918,"(-8117.59, 11.44, 33.89)",8117.66
16 Cyg B b,16 Cyg B,565.737,295.466,50.5168,21.1397,-8119.57,20.4417,25.6228,"(-8119.57, 20.44, 25.62)",8119.64


In [ ]:
rho_0 = 0.0106
R_s = 12500

r_over_Rs = r_galactic / R_s
dark_matter_density = rho_0 / (r_over_Rs * (1 + r_over_Rs)**2)

expl_coord = expl_coord.with_column('dark_matter_density', dark_matter_density)
expl_coord.show(5)

pl_name,hostname,pl_bmasse,ra,dec,sy_dist,galactocentric_x,galactocentric_y,galactocentric_z,coordinate,r_galactic,dark_matter_density
11 Com b,11 Com,4914.9,185.179,17.7933,93.1846,-8123.67,-18.8287,112.047,"(-8123.67, -18.83, 112.05)",8124.47,0.00599067
11 UMi b,11 UMi,4684.81,229.275,71.8239,125.321,-8152.1,89.5176,103.169,"(-8152.1, 89.52, 103.17)",8153.24,0.00595291
14 And b,14 And,1131.15,352.824,39.2358,75.4392,-8141.94,67.5355,-6.24708,"(-8141.94, 67.54, -6.25)",8142.22,0.00596734
14 Her b,14 Her,2559.47,242.602,43.8164,17.9323,-8117.59,11.4422,33.8918,"(-8117.59, 11.44, 33.89)",8117.66,0.00599965
16 Cyg B b,16 Cyg B,565.737,295.466,50.5168,21.1397,-8119.57,20.4417,25.6228,"(-8119.57, 20.44, 25.62)",8119.64,0.00599704


In [ ]:
nan_test = np.isnan(expl_coord.column('pl_bmasse'))
expl_analysis = expl_coord.with_column('nan', nan_test).where('nan', False).drop('nan')
print(expl_coord.num_rows, expl_analysis.num_rows)
expl_analysis

6007 2933


pl_name,hostname,pl_bmasse,ra,dec,sy_dist,galactocentric_x,galactocentric_y,galactocentric_z,coordinate,r_galactic,dark_matter_density
11 Com b,11 Com,4914.9,185.179,17.7933,93.1846,-8123.67,-18.8287,112.047,"(-8123.67, -18.83, 112.05)",8124.47,0.00599067
11 UMi b,11 UMi,4684.81,229.275,71.8239,125.321,-8152.1,89.5176,103.169,"(-8152.1, 89.52, 103.17)",8153.24,0.00595291
14 And b,14 And,1131.15,352.824,39.2358,75.4392,-8141.94,67.5355,-6.24708,"(-8141.94, 67.54, -6.25)",8142.22,0.00596734
14 Her b,14 Her,2559.47,242.602,43.8164,17.9323,-8117.59,11.4422,33.8918,"(-8117.59, 11.44, 33.89)",8117.66,0.00599965
16 Cyg B b,16 Cyg B,565.737,295.466,50.5168,21.1397,-8119.57,20.4417,25.6228,"(-8119.57, 20.44, 25.62)",8119.64,0.00599704
17 Sco b,17 Sco,1373.02,243.462,-11.8378,124.953,-8010.86,2.523,77.9088,"(-8010.86, 2.52, 77.91)",8011.24,0.0061426
18 Del b,18 Del,2926.25,314.608,10.8391,76.222,-8085.32,60.3913,-7.816,"(-8085.32, 60.39, -7.82)",8085.54,0.00604229
1RXS J160929.1-210524 b,1RXS J160929.1-210524,3000,242.376,-21.0831,139.135,-7993.86,-15.9631,72.6614,"(-7993.86, -15.96, 72.66)",7994.2,0.00616593
24 Boo b,24 Boo,280.642,217.156,49.8446,95.9863,-8121.89,47.0545,104.461,"(-8121.89, 47.05, 104.46)",8122.7,0.005993
24 Sex b,24 Sex,632.46,155.868,-0.9024,72.0691,-8143.42,-46.4468,71.5621,"(-8143.42, -46.45, 71.56)",8143.86,0.00596518


In [ ]:
median_density = np.median(expl_analysis.column('dark_matter_density'))
region = ['Galactic Core' if dm >= median_density else 'Galactic Rim'
          for dm in expl_analysis.column('dark_matter_density')]

expl_analysis = expl_analysis.with_column('region', region)
expl_analysis.show(5)

pl_name,hostname,pl_bmasse,ra,dec,sy_dist,galactocentric_x,galactocentric_y,galactocentric_z,coordinate,r_galactic,dark_matter_density,region
11 Com b,11 Com,4914.9,185.179,17.7933,93.1846,-8123.67,-18.8287,112.047,"(-8123.67, -18.83, 112.05)",8124.47,0.00599067,Galactic Rim
11 UMi b,11 UMi,4684.81,229.275,71.8239,125.321,-8152.1,89.5176,103.169,"(-8152.1, 89.52, 103.17)",8153.24,0.00595291,Galactic Rim
14 And b,14 And,1131.15,352.824,39.2358,75.4392,-8141.94,67.5355,-6.24708,"(-8141.94, 67.54, -6.25)",8142.22,0.00596734,Galactic Rim
14 Her b,14 Her,2559.47,242.602,43.8164,17.9323,-8117.59,11.4422,33.8918,"(-8117.59, 11.44, 33.89)",8117.66,0.00599965,Galactic Rim
16 Cyg B b,16 Cyg B,565.737,295.466,50.5168,21.1397,-8119.57,20.4417,25.6228,"(-8119.57, 20.44, 25.62)",8119.64,0.00599704,Galactic Rim


In [ ]:
nan_test = np.isnan(expl_raw.column('pl_bmasse'))
expl_all = expl_raw.with_column('nan', nan_test).where('nan', False).drop('nan')
expl_all

pl_name,hostname,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_radj,pl_radjerr1,pl_radjerr2,pl_radjlim,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_bmasselim,pl_bmassj,pl_bmassjerr1,pl_bmassjerr2,pl_bmassjlim,pl_bmassprov,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_orbeccenlim,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,ttv_flag,st_refname,st_spectype,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_rad,st_raderr1,st_raderr2,st_radlim,st_mass,st_masserr1,st_masserr2,st_masslim,st_met,st_meterr1,st_meterr2,st_metlim,st_metratio,st_logg,st_loggerr1,st_loggerr2,st_logglim,sy_refname,rastr,ra,decstr,dec,sy_dist,sy_disterr1,sy_disterr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate
11 Com b,11 Com,1,2,1,Radial Velocity,2007,Xinglong Station,Published Confirmed,0,<a refstr=TENG_ET_AL__2023 href=https://ui.adsabs.harvar ...,323.21,0.06,-0.05,0,1.178,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,4914.9,39.0929,-39.7286,0,15.464,0.123,-0.125,0,Msini,0.238,0.007,-0.007,0,nan,nan,nan,nan,nan,nan,nan,nan,0,<a refstr=TENG_ET_AL__2023 href=https://ui.adsabs.harvar ...,G8 III,4874,nan,nan,0,13.76,2.85,-2.45,0,2.09,0.64,-0.63,0,-0.26,0.1,-0.1,0,[Fe/H],2.45,0.08,-0.08,0,<a refstr=STASSUN_ET_AL__2019 href=https://ui.adsabs.har ...,12h20m42.91s,185.179,+17d47m35.71s,17.7933,93.1846,1.9238,-1.9238,4.72307,0.023,-0.023,2.282,0.346,-0.346,4.44038,0.0038479,-0.0038479,2023-09-19,2023-08,2023-09-19
11 UMi b,11 UMi,1,1,1,Radial Velocity,2009,Thueringer Landessternwarte Tautenburg,Published Confirmed,0,<a refstr=STASSUN_ET_AL__2017 href=https://ui.adsabs.har ...,516.22,3.2,-3.2,0,1.53,0.07,-0.07,0,nan,nan,nan,nan,nan,nan,nan,nan,4684.81,794.575,-794.575,0,14.74,2.5,-2.5,0,Msini,0.08,0.03,-0.03,0,nan,nan,nan,nan,nan,nan,nan,nan,0,<a refstr=STASSUN_ET_AL__2017 href=https://ui.adsabs.har ...,nan,4213,46,-46,0,29.79,2.84,-2.84,0,2.78,0.69,-0.69,0,-0.02,nan,nan,0,[Fe/H],1.93,0.07,-0.07,0,<a refstr=STASSUN_ET_AL__2019 href=https://ui.adsabs.har ...,15h17m05.90s,229.275,+71d49m26.19s,71.8239,125.321,1.9765,-1.9765,5.013,0.005,-0.005,1.939,0.27,-0.27,4.56216,0.0039035,-0.0039035,2018-09-04,2017-03,2018-09-06
14 And b,14 And,1,1,1,Radial Velocity,2008,Okayama Astrophysical Observatory,Published Confirmed,0,<a refstr=TENG_ET_AL__2023 href=https://ui.adsabs.harvar ...,186.76,0.11,-0.12,0,0.775,0,0,0,nan,nan,nan,nan,nan,nan,nan,nan,1131.15,36.2324,-38.7751,0,3.559,0.114,-0.122,0,Msini,0,nan,nan,0,nan,nan,nan,nan,nan,nan,nan,nan,0,<a refstr=TENG_ET_AL__2023 href=https://ui.adsabs.harvar ...,K0 III,4888,nan,nan,0,11.55,1.12,-0.51,0,1.78,0.43,-0.29,0,-0.21,0.1,-0.1,0,[Fe/H],2.55,0.06,-0.07,0,<a refstr=STASSUN_ET_AL__2019 href=https://ui.adsabs.har ...,23h31m17.80s,352.824,+39d14m09.01s,39.2358,75.4392,0.714,-0.714,5.23133,0.023,-0.023,2.331,0.24,-0.24,4.91781,0.0028262,-0.0028262,2023-09-19,2023-08,2023-09-19
14 Her b,14 Her,1,1,2,Radial Velocity,2002,W. M. Keck Observatory,Published Confirmed,0,<a refstr=FENG_ET_AL__2022 href=https://ui.adsabs.harvar ...,1765.04,1.67709,-1.87256,0,2.774,0.109,-0.12,0,nan,nan,nan,nan,nan,nan,nan,nan,2559.47,518.696,-280.642,0,8.053,1.632,-0.883,0,Mass,0.373,0.008,-0.006,0,nan,nan,nan,nan,nan,nan,nan,nan,0,<a refstr=FENG_ET_AL__2022 href=https://ui.adsabs.harvar ...,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.91,0.113,-0.113,0,nan,nan,nan,nan,nan,nan,nan,nan,nan,<a refstr=STASSUN_ET_AL__2019 href=https://ui.adsabs.har ...,16h10m24.50s,242.602,+43d48m58.90s,43.8164,17.9323,0.0073,-0.0073,6.61935,0.023,-0.023,4.714,0.016,-0.016,6.383,0.0003512,-0.0003512,2023-06-12,2022-09,2023-06-12
16 Cyg B b,16 Cyg B,1,3,1,Radial Velocity,1996,Multiple Observatories,Published Confirmed,0,<a refstr=STASSUN_ET_AL__2017 href=https://u

In [ ]:
# ==============================================================================
# Advanced Statistical Analysis for Exoplanet-Dark Matter Correlations
# ==============================================================================

import pandas as pd
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u
from scipy.stats import spearmanr, linregress, pearsonr, kendalltau
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import statsmodels.api as sm
from statsmodels.formula.api import ols
import warnings
import scipy.stats as stats # Import the stats module

warnings.filterwarnings('ignore')

# Load and prepare data
df = pd.read_csv('expl.csv')
df_clean = df.dropna(subset=['ra', 'dec', 'sy_dist']).copy()

# Coordinate transformation
coords = SkyCoord(
    ra=df_clean['ra'].values*u.deg,
    dec=df_clean['dec'].values*u.deg,
    distance=df_clean['sy_dist'].values*u.pc,
    frame='icrs'
)
gal_coords = coords.transform_to('galactocentric')
df_clean['galactocentric_x'] = gal_coords.x.value
df_clean['galactocentric_y'] = gal_coords.y.value
df_clean['galactocentric_z'] = gal_coords.z.value

# Calculate distance from Galactic Center
df_clean['r_galactic'] = np.sqrt(
    df_clean['galactocentric_x']**2 +
    df_clean['galactocentric_y']**2 +
    df_clean['galactocentric_z']**2
)

# NFW profile parameters
rho_0 = 0.0106  # M_sun/pc^3
R_s = 12500     # parsecs
r_over_Rs = df_clean['r_galactic'] / R_s
df_clean['dark_matter_density'] = rho_0 / (r_over_Rs * (1 + r_over_Rs)**2)
df_clean['log_dm_density'] = np.log10(df_clean['dark_matter_density'])

# Filter planets with known orbital properties
planet_cols = ['pl_orbper', 'pl_orbsmax', 'pl_orbeccen', 'pl_bmasse', 'pl_rade']
df_planets = df_clean.dropna(subset=planet_cols).copy()

print(f"Working with {len(df_planets)} planets with complete properties")

# 1. MULTIVARIATE ANALYSIS
print("=== 1. MULTIVARIATE ANALYSIS ===")

# Create interaction terms and polynomial features
df_planets['dm_r_interaction'] = df_planets['log_dm_density'] * df_planets['r_galactic']
df_planets['dm_squared'] = df_planets['log_dm_density']**2
df_planets['r_squared'] = df_planets['r_galactic']**2

# Standardize variables for better comparison
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df_planets[['log_dm_density', 'r_galactic',
                                                  'pl_orbper', 'pl_bmasse']])
df_planets[['dm_scaled', 'r_scaled', 'period_scaled', 'mass_scaled']] = scaled_features

# Multiple regression
model = ols('period_scaled ~ dm_scaled + r_scaled + mass_scaled + dm_scaled:r_scaled',
            data=df_planets).fit()
print(model.summary())

# 2. STRATIFIED ANALYSIS BY PLANET TYPE
print("\n=== 2. STRATIFIED ANALYSIS BY PLANET TYPE ===")

# Categorize planets by mass
df_planets['mass_category'] = pd.cut(df_planets['pl_bmasse'],
                                    bins=[0, 10, 100, 1000, 10000],
                                    labels=['Earth-like', 'Neptune-like', 'Jupiter-like', 'Super-Jupiter'])

# Categorize by orbital period
df_planets['period_category'] = pd.cut(df_planets['pl_orbper'],
                                      bins=[0, 10, 100, 1000, 10000],
                                      labels=['Short', 'Medium', 'Long', 'Very Long'])

# Analyze correlations within each category
results = []
for mass_cat in df_planets['mass_category'].unique():
    for period_cat in df_planets['period_category'].unique():
        subset = df_planets[(df_planets['mass_category'] == mass_cat) &
                           (df_planets['period_category'] == period_cat)]

        if len(subset) > 20:  # Minimum sample size
            corr, pval = spearmanr(subset['pl_orbper'], subset['log_dm_density'])
            results.append({
                'mass_category': mass_cat,
                'period_category': period_cat,
                'sample_size': len(subset),
                'spearman_r': corr,
                'p_value': pval
            })

results_df = pd.DataFrame(results)
print("Stratified analysis results:")
print(results_df.sort_values('p_value').head(10))

# 3. NON-PARAMETRIC AND ROBUST CORRELATION METHODS
print("\n=== 3. ALTERNATIVE CORRELATION METHODS ===")

# Kendall's tau (more robust to outliers)
tau, pval_tau = kendalltau(df_planets['pl_orbper'], df_planets['log_dm_density'])
print(f"Kendall's tau: {tau:.4f}, p-value: {pval_tau:.4f}")

# Partial correlation (controlling for distance)
def partial_correlation(x, y, z):
    """Calculate partial correlation between x and y controlling for z"""
    x_res = sm.OLS(x, sm.add_constant(z)).fit().resid
    y_res = sm.OLS(y, sm.add_constant(z)).fit().resid
    return pearsonr(x_res, y_res)[0]

part_corr = partial_correlation(df_planets['pl_orbper'],
                               df_planets['log_dm_density'],
                               df_planets['r_galactic'])
print(f"Partial correlation (controlling for distance): {part_corr:.4f}")

# 4. DIMENSIONALITY REDUCTION AND CLUSTERING
print("\n=== 4. CLUSTERING ANALYSIS ===")

# Prepare data for clustering
cluster_data = df_planets[['log_dm_density', 'r_galactic', 'pl_orbper', 'pl_bmasse']].dropna()
cluster_data_scaled = StandardScaler().fit_transform(cluster_data)

# DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=10)
clusters = dbscan.fit_predict(cluster_data_scaled)
df_planets.loc[cluster_data.index, 'cluster'] = clusters

# Analyze cluster characteristics
if len(np.unique(clusters)) > 1:
    cluster_stats = df_planets.groupby('cluster').agg({
        'log_dm_density': ['mean', 'std'],
        'pl_orbper': ['mean', 'std'],
        'pl_bmasse': ['mean', 'std'],
        'r_galactic': ['mean', 'std']
    }).round(3)
    print("Cluster characteristics:")
    print(cluster_stats)

# 5. BAYESIAN APPROACH
print("\n=== 5. BAYESIAN CORRELATION ANALYSIS ===")

# Bayesian correlation estimation (simplified)
def bayesian_correlation(x, y, n_iter=1000):
    """Simple Bayesian correlation estimation using bootstrap"""
    corrs = []
    for _ in range(n_iter):
        sample_idx = np.random.choice(len(x), size=len(x), replace=True)
        x_sample = x.iloc[sample_idx]
        y_sample = y.iloc[sample_idx]
        if len(x_sample) > 10:
            corr, _ = spearmanr(x_sample, y_sample)
            if not np.isnan(corr):
                corrs.append(corr)

    return np.mean(corrs), np.std(corrs)

bayes_mean, bayes_std = bayesian_correlation(df_planets['pl_orbper'],
                                           df_planets['log_dm_density'])
print(f"Bayesian correlation estimate: {bayes_mean:.4f} ± {bayes_std:.4f}")

# 6. SPATIAL AUTOCORRELATION ANALYSIS
print("\n=== 6. SPATIAL AUTOCORRELATION ===")

# Check if there's spatial clustering in residuals
model_residuals = sm.OLS(df_planets['pl_orbper'],
                        sm.add_constant(df_planets[['log_dm_density', 'r_galactic']])).fit().resid

# Moran's I (simplified calculation)
def morans_i(residuals, coords):
    """Calculate simplified Moran's I for spatial autocorrelation"""
    n = len(residuals)
    mean_res = np.mean(residuals)

    # Calculate spatial weights (inverse distance)
    dist_matrix = np.sqrt(np.sum((coords[:, np.newaxis] - coords)**2, axis=2))
    np.fill_diagonal(dist_matrix, np.inf)  # Exclude self
    weights = 1 / dist_matrix
    weights[np.isinf(weights)] = 0 # Handle infinite weights
    weights = weights / np.sum(weights)

    # Calculate Moran's I
    numerator = np.sum(weights * (residuals - mean_res)[:, np.newaxis] * (residuals - mean_res))
    denominator = np.sum((residuals - mean_res)**2)

    # Handle case where denominator is zero
    if denominator == 0:
        return np.nan
    else:
        return (n / np.sum(weights)) * (numerator / denominator)


# Use galactic coordinates for spatial analysis
coords_array = df_planets[['galactocentric_x', 'galactocentric_y', 'galactocentric_z']].values
moran_i = morans_i(model_residuals.values, coords_array)
print(f"Moran's I (spatial autocorrelation): {moran_i:.4f}")

# 7. VISUALIZATION OF POTENTIAL RELATIONSHIPS
print("\n=== 7. ADVANCED VISUALIZATION ===")

plt.figure(figsize=(15, 10))

# 3D scatter plot
ax = plt.subplot(2, 2, 1, projection='3d')
scatter = ax.scatter(df_planets['r_galactic'], df_planets['log_dm_density'],
                    df_planets['pl_orbper'], c=df_planets['pl_bmasse'],
                    cmap='viridis', alpha=0.6)
ax.set_xlabel('Galactic Radius (pc)')
ax.set_ylabel('log(DM Density)')
ax.set_zlabel('Orbital Period (days)')
plt.colorbar(scatter, label='Planet Mass (Earth masses)')

# Hexbin plot to show density of points
plt.subplot(2, 2, 2)
hb = plt.hexbin(df_planets['log_dm_density'], df_planets['pl_orbper'],
                gridsize=30, cmap='Blues', mincnt=1)
plt.xlabel('log(DM Density)')
plt.ylabel('Orbital Period (days)')
plt.colorbar(hb, label='Count')

# Conditional plot by mass category
plt.subplot(2, 2, 3)
for mass_cat in df_planets['mass_category'].unique():
    subset = df_planets[df_planets['mass_category'] == mass_cat]
    if len(subset) > 30:
        plt.scatter(subset['log_dm_density'], subset['pl_orbper'],
                   alpha=0.6, label=mass_cat, s=30)
plt.xlabel('log(DM Density)')
plt.ylabel('Orbital Period (days)')
plt.legend()
plt.yscale('log')

# Residual plot
plt.subplot(2, 2, 4)
plt.scatter(df_planets['log_dm_density'], model_residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('log(DM Density)')
plt.ylabel('Residuals (Orbital Period)')

plt.tight_layout()
plt.show()

# 8. POWER ANALYSIS
print("\n=== 8. POWER ANALYSIS ===")

# Calculate minimum detectable effect size
def power_analysis(corr, n, alpha=0.05):
    """Calculate power for correlation analysis"""
    # Convert correlation to effect size
    effect_size = np.abs(corr)

    # Calculate non-centrality parameter
    ncp = effect_size * np.sqrt(n - 2) / np.sqrt(1 - effect_size**2)

    # Critical t-value
    t_crit = stats.t.ppf(1 - alpha/2, n-2)

    # Power
    power = 1 - stats.t.cdf(t_crit, n-2, ncp) + stats.t.cdf(-t_crit, n-2, ncp)
    return power

current_power = power_analysis(0.07, len(df_planets))
print(f"Power to detect r=0.07 with n={len(df_planets)}: {current_power:.3f}")

# Calculate minimum detectable effect
for effect_size in [0.1, 0.15, 0.2]:
    power = power_analysis(effect_size, len(df_planets))
    print(f"Power to detect r={effect_size}: {power:.3f}")

print("\n=== RECOMMENDATIONS ===")
print("1. The weak correlations (r ≈ 0.07) suggest any DM effect is very subtle")
print("2. Focus on specific planet subtypes where effects might be stronger")
print("3. Consider system-level properties rather than individual planets")
print("4. The relationship might be non-linear - try polynomial or spline models")
print("5. Spatial autocorrelation should be accounted for in any analysis")



import plotly.express as px

# 1. Scatter of Orbital Period vs log(DM Density), colored by planet mass
fig1 = px.scatter(df_planets,
                  x='log_dm_density',
                  y='pl_orbper',
                  color='mass_category',
                  size='pl_bmasse',
                  hover_data=['pl_rade', 'pl_orbper', 'pl_bmasse', 'cluster'],
                  title='Orbital Period vs Log(Dark Matter Density) by Planet Mass',
                  labels={'log_dm_density':'Log(DM Density)','pl_orbper':'Orbital Period (days)'}
                 )
fig1.update_yaxes(type='log')
fig1.show()

# 2. 3D Scatter: Galactic Radius, log(DM Density), Orbital Period
fig2 = px.scatter_3d(df_planets,
                     x='r_galactic',
                     y='log_dm_density',
                     z='pl_orbper',
                     color='mass_category',
                     size='pl_bmasse',
                     hover_data=['pl_rade', 'pl_orbper', 'pl_bmasse', 'cluster'],
                     title='3D: Galactic Radius vs DM Density vs Orbital Period'
                    )
fig2.update_layout(scene=dict(zaxis_type='log'))
fig2.show()

# 3. Boxplot of Orbital Period by Mass Category
fig3 = px.box(df_planets,
              x='mass_category',
              y='pl_orbper',
              log_y=True,
              points="all",
              title="Distribution of Orbital Period by Planet Mass Category",
              labels={'pl_orbper':'Orbital Period (days)', 'mass_category':'Planet Mass Category'})
fig3.show()

# 4. Density heatmap of DM vs Orbital Period
fig4 = px.density_heatmap(df_planets,
                          x='log_dm_density',
                          y='pl_orbper',
                          nbinsx=40,
                          nbinsy=40,
                          color_continuous_scale='Viridis',
                          title='Density of Planets: DM Density vs Orbital Period')
fig4.update_yaxes(type='log')
fig4.show()

Working with 1914 planets with complete properties
=== 1. MULTIVARIATE ANALYSIS ===
                            OLS Regression Results                            
Dep. Variable:          period_scaled   R-squared:                       0.090
Model:                            OLS   Adj. R-squared:                  0.089
Method:                 Least Squares   F-statistic:                     47.47
Date:                Sun, 21 Sep 2025   Prob (F-statistic):           4.27e-38
Time:                        14:36:29   Log-Likelihood:                -2625.1
No. Observations:                1914   AIC:                             5260.
Df Residuals:                    1909   BIC:                             5288.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------